# BAX Emittance Minimization (LCLS-II)
In this notebook, we hope to demonstrate Bayesian Algorithm Execution (BAX) in Xopt as a means of minimizing the emittance at LCLS-II. BAX is a generalization of Bayesian Optimization that seeks to acquire observations that provide our model with maximal information about our property of interest. In this example, our property of interest is the minimal emittance and its location in tuning-parameter-space. See https://arxiv.org/pdf/2209.04587.pdf for details.

In [ ]:
# set up env for running on SLAC production servers
import os
os.environ['OMP_NUM_THREADS']=str(6)

In [ ]:
run_dir = '/home/physics/ml_tuning/20240225_LCLS_II_Injector'

# Read pv info from YAML files

In [ ]:
import sys
import yaml
sys.path.append("../../")
sys.path.append("../../../")

from common import get_pv_objects, save_reference_point, set_magnet_strengths, \
    measure_pvs

In [ ]:
pv_bounds = yaml.safe_load(open("../../pv_bounds.yml"))
pv_objects = get_pv_objects("../../tracked_pvs.yml")

In [ ]:
pv_bounds

# load reference point
Also define a function to write the reference values to the pvs

In [ ]:
reference = yaml.safe_load(open("../../reference.yml"))

def reset_pvs():
    set_magnet_strengths(reference, pv_objects, validate=False)

In [ ]:
reset_pvs()

In [ ]:
from scripts.image import ImageDiagnostic
import yaml

fname = 'otr_diagnostic.yml' #run_dir + "OTRS_HTR_330_config.yml"

image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))

image_diagnostic.min_log_intensity = 3.0
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
print(image_diagnostic.yaml())


In [ ]:
image_diagnostic.measure_background()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image_diagnostic.background_image)

In [ ]:
image_diagnostic.resolution

In [ ]:
from epics import caget

In [ ]:
caget('OTRS:HTR:330:RESOLUTION')

In [ ]:
image_diagnostic.test_measurement()

# Imports

In [ ]:
# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")
import torch
from xopt import Xopt
from xopt.vocs import VOCS
from xopt.evaluator import Evaluator
import numpy as np
import random

# General settings

In [ ]:
# general settings



# random seeds for reproducibility
rand_seed = 2
torch.manual_seed(rand_seed)
np.random.seed(rand_seed) # only(?) affects initial random observations through Xopt 
random.seed(rand_seed)

# Evaluator

In [ ]:
import time
# define function to measure the total size on OTR4
def eval_beamsize(inputs):
    
    # set pvs and wait for BACT to settle to correct values (validate=True)
    set_magnet_strengths(inputs, pv_objects, validate=False)
    time.sleep(2)
    # measure all pvs - except for names in inputs
    results = measure_pvs(
        [name for name in pv_objects.keys() if name not in inputs], pv_objects 
    )

    # do some calculations
    results["time"] = time.time()

    # add beam size measurement to results dict
    beamsize_results = image_diagnostic.measure_beamsize()
    results["Sx_mm"] = beamsize_results["Sx"] * 1e-2
    results["Sy_mm"] = beamsize_results["Sy"] * 1e-2

    #add beam size squared (mm^2)
    results["xrms_sq"] = results["Sx_mm"]**2
    results["yrms_sq"] = results["Sy_mm"]**2
    results = beamsize_results | results
    return results

evaluator = Evaluator(function=eval_beamsize)


In [ ]:
eval_beamsize({})

## VOCS

In [ ]:
# create Xopt objects
from xopt import VOCS

IMAGE_CONSTRAINTS = {
            "bb_penalty": ["LESS_THAN", 0.0],
        }

vocs = VOCS(
    variables = {
        'SOLN:GUNB:212:BCTRL': [0.044, 0.044574],
        'QUAD:HTR:120:BCTRL': [-4.46919, 4.4792]
    },
    constraints = IMAGE_CONSTRAINTS,
    observables = ["xrms_sq", "yrms_sq"],
)
vocs.variable_names

# Run Bayesian Exploration on a grid

In [ ]:
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from xopt.generators.bayesian.bayesian_exploration import BayesianExplorationGenerator
from xopt.numerical_optimizer import GridOptimizer
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from copy import deepcopy
sys.path.append("../../emitopt/")

meas_dim = 0
tuning_dims = [1]

covar_module = (MaternKernel(ard_num_dims=len(tuning_dims),
                              active_dims=tuning_dims,
                              lengthscale_prior=None) *
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )
scaled_covar_module = ScaleKernel(covar_module)

# prepare options for Xopt generator
covar_module_dict = {
    'xrms_sq': scaled_covar_module,
    "yrms_sq": deepcopy(scaled_covar_module)
}
# covar_module_dict = {}
model_constructor = StandardModelConstructor(
    covar_modules=covar_module_dict, use_low_noise_prior=True
)

generator = BayesianExplorationGenerator(
    vocs=vocs,
    gp_constructor=model_constructor,
    numerical_optimizer=GridOptimizer(n_grid_points=10)
)

from xopt import Xopt
X_bayes_exp = Xopt(
    vocs=vocs,
    generator=generator,
    evaluator=evaluator,
    strict=True
)

## sample in local area around reference point

In [ ]:
from epics import caget_many
from xopt.utils import get_local_region

reset_pvs()
current_value = dict(
    zip(
        X_bayes_exp.vocs.variable_names,
        caget_many(X_bayes_exp.vocs.variable_names)
    )
)
print(current_value)


random_sample_region = get_local_region(current_value,X_bayes_exp.vocs, fraction=0.1)
X_bayes_exp.random_evaluate(10, custom_bounds=random_sample_region)

In [ ]:
for i in range(40):
    X_bayes_exp.step()

## Visualize model/data

In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
fig,ax = visualize_generator_model(
    X_bayes_exp.generator,
    variable_names=['SOLN:GUNB:212:BCTRL','QUAD:HTR:120:BCTRL'],
    output_names=["xrms_sq","yrms_sq","bb_penalty"],
)


# Run BAX

In [ ]:
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from copy import deepcopy
sys.path.append("../../emitopt/")
import torch
from emitopt.algorithms import ScipyMinimizeEmittanceXY

meas_dim = 0
tuning_dims = [1]

covar_module = (MaternKernel(ard_num_dims=len(tuning_dims),
                              active_dims=tuning_dims,
                              lengthscale_prior=None) *
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )
scaled_covar_module = ScaleKernel(covar_module)

# prepare options for Xopt generator
covar_module_dict = {'Sx_squared': scaled_covar_module, "Sy_squared": deepcopy(scaled_covar_module)}
model_constructor = StandardModelConstructor(
    covar_modules=covar_module_dict, use_low_noise_prior=True
)

QUAD_LENGTH = 0.124 # m
RMAT_X =  [
    [-1.4591, 2.9814],
    [-0.4998, 0.3358]
]
RMAT_Y = [
    [4.8650, 8.5737],
    [-0.1597, -0.0758]
]
THICK_QUAD = False
BEAM_ENERGY = 0.088 # GeV
SCALE_FACTOR = 2.74 # multiplicative factor to convert from measurement quad PV units to geometric focusing strength

algo_kwargs = {
        'x_key': "xrms_sq",
        'y_key': "yrms_sq",
        'scale_factor': SCALE_FACTOR,
        'q_len': QUAD_LENGTH,
        'rmat_x': torch.tensor(RMAT_X),
        'rmat_y': torch.tensor(RMAT_Y),
        'n_samples': 10,
        'meas_dim': [0],
        'n_steps_measurement_param': 11,
        'thick_quad': THICK_QUAD
        }
algo = ScipyMinimizeEmittanceXY(**algo_kwargs)

# Initialize Xopt Optimizer

In [ ]:
from xopt.generators.bayesian.bax_generator import BaxGenerator


#construct BAX generator
generator = BaxGenerator(
    vocs=vocs,
    gp_constructor=model_constructor,
    algorithm=algo,
    numerical_optimizer=GridOptimizer(n_grid_points=10)
)

#construct Xopt optimizer and add data
X_bax = Xopt(
    evaluator=evaluator,
    generator=generator,
    vocs=vocs,
    dump_file="BAX_run_be.yml"
)
X_bax.add_data(X_bayes_exp.data)

In [ ]:
for i in range(10):
    X_bax.step()

## view data and model

In [ ]:
from emitopt.plot_utils import plot_virtual_emittance_vs_tuning
plot_virtual_emittance_vs_tuning(X_bax, torch.tensor([[0.0,450.,200]]), n_points=50)

In [ ]:
from emitopt.plot_utils import plot_sample_optima_convergence_inputs
fig, axs = plot_sample_optima_convergence_inputs(results, show_valid_only=False)

In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
visualize_generator_model(
    X_bax.generator,
    variable_names=['QUAD:HTR:120:BCTRL']
)
